In [1]:
import pandas as pd 
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tokenize(headline):
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model=AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    calssifier=pipeline('text-classification',model=model,tokenizer=tokenizer)
    res=calssifier(headline)
    return res[0]

In [3]:
file_path = r'sp500_companies.csv'
data1 = pd.read_csv(file_path)
print(data1.head())
symbols_array = data1['Symbol'].tolist()

  Exchange Symbol              Shortname               Longname  \
0      NMS   AAPL             Apple Inc.             Apple Inc.   
1      NMS   NVDA     NVIDIA Corporation     NVIDIA Corporation   
2      NMS   MSFT  Microsoft Corporation  Microsoft Corporation   
3      NMS   GOOG          Alphabet Inc.          Alphabet Inc.   
4      NMS  GOOGL          Alphabet Inc.          Alphabet Inc.   

                   Sector                        Industry  Currentprice  \
0              Technology            Consumer Electronics        235.00   
1              Technology                  Semiconductors        138.00   
2              Technology       Software - Infrastructure        418.16   
3  Communication Services  Internet Content & Information        165.05   
4  Communication Services  Internet Content & Information        163.42   

       Marketcap        Ebitda  Revenuegrowth           City State  \
0  3572963475456  1.317810e+11          0.049      Cupertino    CA   
1  338

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax


In [5]:
model_name = "ProsusAI/finbert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
def get_sentiment(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probabilities = softmax(logits, dim=1).tolist()[0]
    
    # FinBERT has three sentiment classes: positive, neutral, and negative
    labels = ["negative", "neutral", "positive"]
    sentiment_class = labels[probabilities.index(max(probabilities))]
    sentiment_score = max(probabilities)  # Confidence score
    
    return sentiment_class, sentiment_score

In [11]:

def generate_sentiment():
    for symbol in symbols_array:
        filename = f'news_data/{symbol}.csv'
        df = pd.read_csv(filename)
        if df.empty:
            continue

        new_df = pd.DataFrame(columns=['datetime', 'title', 'source', 'link', 'sentiment_class', 'sentiment_score'])
        rows = []
        for index, row in df.iterrows():
            headline = row['title']
            sentiment_class, sentiment_score = get_sentiment(headline)

            rows.append({
                'datetime': row['datetime'],
                'title': row['title'],
                'source': row['source'],
                'link': row['link'],
                'sentiment_class': sentiment_class,
                'sentiment_score': sentiment_score
            })

        new_df = pd.concat([new_df, pd.DataFrame(rows)], ignore_index=True)

        new_df.to_csv(f'sentiment_data/{symbol}.csv')


In [ ]:
generate_sentiment()

C:\Users\ahmed\AppData\Local\Temp\ipykernel_3808\786632027.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame(rows)], ignore_index=True)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_3808\786632027.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame(rows)], ignore_index=True)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_3808\786632027.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-N